In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

In [ ]:
df = pd.read_csv('Data_In.csv')
df.rename(columns = {'company_name' : 'Company'}, inplace = True)

# Print Rows with Null Values
# df[df.isnull().any(axis=1)]

In [ ]:
search_term = 'AMAZON'
factor = 'Company'

# Filter rows where 'Type' is equal to 'Health'
filtered_df = df[df[factor] == search_term]


In [ ]:
data = filtered_df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [ ]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length, 0]
        label = data[i + seq_length, 0]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [ ]:
seq_length = 5
n_samples = len(data_scaled)
sequences, labels = create_sequences(data_scaled, seq_length)
split = int(0.8 * len(sequences))
X_test, y_test = sequences[split:], labels[split:]
X_train, y_train = sequences[:split], labels[:split]

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(100, activation='relu', input_shape=(seq_length, 1), return_sequences=True))
model_lstm.add(LSTM(50, activation='relu'))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
model_lstm.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))
predictions_lstm = model_lstm.predict(X_test)

In [ ]:
predictions__LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
dates_test = df['Date'].iloc[split + seq_length:]
# df_plot_lstm = pd.DataFrame({'Fechas': dates_test, 'Real': y_test_actual_LSTM.flatten(), 'Predict': predictions__LSTM.flatten()})
df_plot_lstm = pd.DataFrame({'Fechas': dates_test[:len(predictions__LSTM)], 'Real': y_test_actual_LSTM.flatten(), 'Predict': predictions__LSTM.flatten()})

df_plot_lstm.tail(7)
# df_plot_lstm.to_csv('AMAZON.csv')